## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
apptype_value_counts = application_df["APPLICATION_TYPE"].value_counts()
apptype_value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(apptype_value_counts[apptype_value_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df["CLASSIFICATION"].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts[classification_value_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_value_counts[classification_value_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_application_df = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
y = converted_application_df['IS_SUCCESSFUL'].values
X = converted_application_df.drop(columns=['IS_SUCCESSFUL']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model - 1st Attempt

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# 1st Attempt - adding another hidden layer

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 80)                3520      
                                                                 
 dense_25 (Dense)            (None, 30)                2430      
                                                                 
 dense_26 (Dense)            (None, 10)                310       
                                                                 
 dense_27 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6271 (24.50 KB)
Trainable params: 6271 (24.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100


804/804 [==============================] - 2s 1ms/step - loss: 0.5717 - accuracy: 0.7188
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5546 - accuracy: 0.7295
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7279
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5503 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5489 - accuracy: 0.7317
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7327
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5464 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5469 - accuracy: 0.7331
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5452 - accuracy: 0.7331
E

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5593 - accuracy: 0.7284 - 317ms/epoch - 1ms/step
Loss: 0.5592710375785828, Accuracy: 0.728396475315094


## Compile, Train and Evaluate the Model - 2nd Attempt

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# 2nd Attempt - using a different activation function for the hidden layers

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = number_input_features, activation="tanh")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 80)                3520      
                                                                 
 dense_29 (Dense)            (None, 30)                2430      
                                                                 
 dense_30 (Dense)            (None, 10)                310       
                                                                 
 dense_31 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6271 (24.50 KB)
Trainable params: 6271 (24.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 989us/step - loss: 0.5698 - accuracy: 0.7202
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5559 - accuracy: 0.7287
Epoch 3/100
804/804 [==============================] - 1s 922us/step - loss: 0.5529 - accuracy: 0.7310
Epoch 4/100
804/804 [==============================] - 1s 977us/step - loss: 0.5507 - accuracy: 0.7315
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7321
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7328
Epoch 7/100
804/804 [==============================] - 1s 958us/step - loss: 0.5466 - accuracy: 0.7324
Epoch 8/100
804/804 [==============================] - 1s 996us/step - loss: 0.5454 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 1s 963us/step - loss: 0.5451 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5578 - accuracy: 0.7293 - 272ms/epoch - 1ms/step
Loss: 0.5577701926231384, Accuracy: 0.7293294668197632


## Compile, Train and Evaluate the Model - 3rd Attempt

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# 3rd Attempt - adding more epochs to the training regimen

number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim = number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# # Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_51 (Dense)            (None, 80)                3520      
                                                                 
 dense_52 (Dense)            (None, 30)                2430      
                                                                 
 dense_53 (Dense)            (None, 10)                310       
                                                                 
 dense_54 (Dense)            (None, 1)                 11        
                                                                 
Total params: 6271 (24.50 KB)
Trainable params: 6271 (24.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5691 - accuracy: 0.7171
Epoch 2/200
804/804 [==============================] - 1s 944us/step - loss: 0.5553 - accuracy: 0.7276
Epoch 3/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5521 - accuracy: 0.7294
Epoch 4/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7312
Epoch 5/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5489 - accuracy: 0.7326
Epoch 6/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7321
Epoch 7/200
804/804 [==============================] - 1s 990us/step - loss: 0.5473 - accuracy: 0.7334
Epoch 8/200
804/804 [==============================] - 1s 916us/step - loss: 0.5463 - accuracy: 0.7343
Epoch 9/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5461 - accuracy: 0.7338
Epoch 10/200
804/804 [==============================] - 1s 1ms/step - loss: 0.5453 - 

In [62]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5584 - accuracy: 0.7300 - 284ms/epoch - 1ms/step
Loss: 0.5584006309509277, Accuracy: 0.7300291657447815


In [63]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

c:\Users\iahme\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
